# PCA-LDA

Given a collection of of n m-dimensional samples stored in the columns of matrix $X = [x_1 , . . . , x_n ]$. We also assume that data are centered. Otherwise, we can always substract the mean $m = \frac{1}{n} \sum_{i=1}^{n}x_i$.

An efficient way of doing this is:

\begin{align*}
X − M = X \left(I − \frac{1}{n}1_n1_n^T \right)
\end{align*}

## Statistical perspective
We want to find the vector $w$ such that the variance of the projected features $y_i = w_i^Tx_i$ is maximised.The variance can be expressed:

\begin{align*}
\sigma_y^2 = \frac{1}{n} \sum_{i=1}^{n}(y_i - \mu_y)
\end{align*}

Where $\mu_y$ is the mean. Since we supposed the data are centered, $\mu_y = 0$.

The maximiser is:

\begin{align*}
w^* & = \mathrm{argmax}_{w} \frac{1}{2n} \sum_{i=1}^{n}(w^Tx_i)^2 = \mathrm{argmax}_{w}  \frac{1}{2n} \sum_{i=1}^{n}w^Tx_i x_i^Tw\\
& = \mathrm{argmax}_{w} \frac{1}{2} w^T\frac{XX^T}{n}w  = \mathrm{argmax}_{w} \frac{1}{2} w^T S_tw  
\end{align*}

Where $S_t = \frac{XX^T}{n}$ is called the covariance matrix (or total-scatter matrix). We add the following constraint not to end up with a trivial solution $w=\infty$.

\begin{align*}
w^*  &= \mathrm{argmax}_{w} \frac{1}{2} w^T S_tw \\
\text{subject to } &w^Tw=1
\end{align*}

The corresponding Lagrangian is:

\begin{align*}
L(w, \lambda) = \frac{1}{2}w^TS_tw - \lambda(w^Tw-1)
\end{align*}

Taking the derivative:
\begin{align*}
\frac{ \partial L}{\partial w} = S_tw - \lambda w = 0
\end{align*}

This means $w$ is an eigenvector of $S_t$, and $\lambda$ the corresponding eigenvalue. Plugging back this expression in the initial problem:

\begin{align*}
\lambda^* = \mathrm{argmax}_{\lambda} \lambda
\end{align*}

So the largest eigenvalue is chosen.

This reasonning can be applied for $y_i \in \mathbb{R}^d$:

\begin{align*}
W^* =& \mathrm{argmax}_{W} \frac{1}{2n} \sum_{k=1}^{d} \sum_{i=1}^{n} y_{ki}^2  = \mathrm{argmax}_{W} \frac{1}{2n} \sum_{k=1}^{d} \sum_{i=1}^{n} (w_k^Tx_i)^2 \\
& = \mathrm{argmax}_{W} \frac{1}{2n} \sum_{k=1}^{d} \sum_{i=1}^{n} w_k^Tx_ix_i^Tw_k \\
& = \mathrm{argmax}_{W} \frac{1}{2n} \sum_{k=1}^{d}  w_k^T \left(\sum_{i=1}^{n}x_ix_i^T \right)w_k \\
& = \mathrm{argmax}_{W} \frac{1}{2} \sum_{k=1}^{d}  w_k^T S_t w_k \\
& = \mathrm{argmax}_{W} \frac{1}{2} \mathrm{Tr} (W^T S_t W) \\
\end{align*}

So the general problem is:

\begin{align*}
\max_W \mathrm{Tr} (W^T S_tW) \\
\text{subject to } &W^TW=1
\end{align*}

Taking the partial derivative of the Lagrangian leads to a similar condition:

\begin{align*}
S_tW=W\Lambda
\end{align*}

Where $\Lambda$ is the matrix of the Lagrange multipliers.

Assuming the eigendecomposition of $S_t$ is $S_t = U \Lambda U^T$, then $W = U_d = [u_1, ..., u_d]$.

Similarly as above, the cost function can be written as:

\begin{align*}
    \mathrm{Tr}(W^TS_TW) = \mathrm{Tr}(W^T U \Lambda U^T W) = \mathrm{Tr}(\Lambda_d) = \sum_{k=1}^{d} \lambda_k
\end{align*}

Since $\lambda_k \ge 0$, maximisation of the above is equivalent to take the $d$ largest eigenvalues.

In [ ]:
#load dataset